In [76]:
import pandas as pd
import os
import re
import plotly.graph_objects as go
import plotly.express as px

## Quellen
- [Link zum Download der Remit Dateien](https://bmrs.elexon.co.uk/remit?timeRangeType=Custom&timeRangeLength=Custom&assetID=T_HOWAO-1&endpoint=published&revisionHistory=include-history&from=2024-01-01T07%3A30%3A00.000Z&to=2025-01-01T07%3A30%3A00.000Z&messageType=Unavailabilities+of+Electricity+Facilities%2COther+Market+Information&unavailabilityType=Planned%2CUnplanned&eventType=Production+Unavailability%2CTransmission+Unavailability%2CConsumption+Unavailability%2COther+Unavailability)

- [Dokumenation BMRS API and Data Push User Guide](https://assets.elexon.co.uk/wp-content/uploads/2018/09/28160411/BMRS-API-Data-Push-User-Guide.pdf)

In [77]:
#  # Liest alle JSON-Dateien aus einem angegebenen Ordner ein und wandelt sie in CSV-Dateien um
#  # Ordnerpfad zu den JSON-Dateien
# json_folder = 'data/Json_Dateien/'

# # Alle JSON-Dateien im Ordner einlesen und in CSV umwandeln
# for filename in os.listdir(json_folder):
#     if filename.endswith('.json'):
#         json_file = os.path.join(json_folder, filename)  # Vollständiger Pfad zur JSON-Datei
#         data = pd.read_json(json_file)

#         # CSV-Dateiname erstellen
#         csv_file = os.path.join(json_folder, filename.replace('.json', '.csv'))  # Pfad zur gewünschten CSV-Datei
#         data.to_csv(csv_file, index=False)

#         print(f'Die JSON-Datei {filename} wurde erfolgreich in {csv_file} umgewandelt.')

In [78]:
# Ordnerpfad zu den CSV-Dateien
csv_folder = 'data/Json_Dateien'

# Leere Liste zum Speichern der DataFrames
dataframes = []

# Alle CSV-Dateien im Ordner einlesen
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_folder, filename)  # Vollständiger Pfad zur CSV-Datei
        df = pd.read_csv(csv_file)  # Einlesen der CSV-Datei
        dataframes.append(df)  # Hinzufügen des DataFrames zur Liste

# Alle DataFrames zu einem einzigen DataFrame zusammenführen
df = pd.concat(dataframes, ignore_index=True)

In [79]:
df.head(2)

,id,dataset,mrid,revisionNumber,publishTime,createdTime,messageType,messageHeading,eventType,unavailabilityType,...,fuelType,normalCapacity,availableCapacity,unavailableCapacity,eventStatus,eventStartTime,eventEndTime,cause,relatedInformation,outageProfile
0,60924,REMIT,11XDONG-PT-----2-NGET-RMT-00001020,2,2020-11-26T13:54:00Z,2020-11-26T13:54:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-11-26T09:00:00Z,2020-11-26T18:00:00Z,Planned Outage,HOW01 Z12 Dry run interlink test,"[{'startTime': '2020-11-26T09:00:00Z', 'endTim..."
1,185054,REMIT,11XDONG-PT-----2-NGET-RMT-00001024,2,2020-12-10T16:12:00Z,2020-12-10T16:12:00Z,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,Active,2020-12-11T06:00:00Z,2020-12-11T16:00:00Z,Planned Outage,HOW01 BMU 2 will be undergoing frequency respo...,"[{'startTime': '2020-12-11T06:00:00Z', 'endTim..."


In [80]:
df.columns

Index(['id', 'dataset', 'mrid', 'revisionNumber', 'publishTime', 'createdTime',
       'messageType', 'messageHeading', 'eventType', 'unavailabilityType',
       'participantId', 'registrationCode', 'assetId', 'assetType',
       'affectedUnit', 'affectedUnitEIC', 'affectedArea', 'biddingZone',
       'fuelType', 'normalCapacity', 'availableCapacity',
       'unavailableCapacity', 'eventStatus', 'eventStartTime', 'eventEndTime',
       'cause', 'relatedInformation', 'outageProfile'],
      dtype='object')

In [81]:
# Konvertiere die Spalten 'publishTime' und 'createdTime' in Datetime-Objekte
df['publishTime'] = pd.to_datetime(df['publishTime'])
df['createdTime'] = pd.to_datetime(df['createdTime'])

# Versuche, 'eventStatus' und 'eventStartTime' in Datetime-Objekte zu konvertieren; ungültige Werte werden in NaT umgewandelt
df['eventStatus'] = pd.to_datetime(df['eventStatus'], errors='coerce')
df['eventStartTime'] = pd.to_datetime(df['eventStartTime'], errors='coerce')

C:\Users\Michael Jäckle\AppData\Local\Temp\ipykernel_31700\4078416652.py:6: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [82]:
df.dtypes

id                                   int64
dataset                             object
mrid                                object
revisionNumber                       int64
publishTime            datetime64[ns, UTC]
createdTime            datetime64[ns, UTC]
messageType                         object
messageHeading                      object
eventType                           object
unavailabilityType                  object
participantId                       object
registrationCode                    object
assetId                             object
assetType                           object
affectedUnit                        object
affectedUnitEIC                     object
affectedArea                        object
biddingZone                         object
fuelType                            object
normalCapacity                       int64
availableCapacity                    int64
unavailableCapacity                  int64
eventStatus                 datetime64[ns]
eventStartT

In [83]:
df.head(2)

,id,dataset,mrid,revisionNumber,publishTime,createdTime,messageType,messageHeading,eventType,unavailabilityType,...,fuelType,normalCapacity,availableCapacity,unavailableCapacity,eventStatus,eventStartTime,eventEndTime,cause,relatedInformation,outageProfile
0,60924,REMIT,11XDONG-PT-----2-NGET-RMT-00001020,2,2020-11-26 13:54:00+00:00,2020-11-26 13:54:00+00:00,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,NaT,2020-11-26 09:00:00+00:00,2020-11-26T18:00:00Z,Planned Outage,HOW01 Z12 Dry run interlink test,"[{'startTime': '2020-11-26T09:00:00Z', 'endTim..."
1,185054,REMIT,11XDONG-PT-----2-NGET-RMT-00001024,2,2020-12-10 16:12:00+00:00,2020-12-10 16:12:00+00:00,UnavailabilitiesOfElectricityFacilities,REMIT Information,Production unavailability,Planned,...,Wind Offshore,400,0,400,NaT,2020-12-11 06:00:00+00:00,2020-12-11T16:00:00Z,Planned Outage,HOW01 BMU 2 will be undergoing frequency respo...,"[{'startTime': '2020-12-11T06:00:00Z', 'endTim..."


In [84]:
# Funktion zum Extrahieren der Werte aus dem Dictionary
def extract_values(outage_profile):
    # Überprüfen, ob der Wert in der Spalte gültig ist
    if isinstance(outage_profile, str):
        # Suchen nach allen startTime, endTime und capacity Werten
        start_times = re.findall(r"'startTime': '(.*?)'", outage_profile)
        end_times = re.findall(r"'endTime': '(.*?)'", outage_profile)
        capacities = re.findall(r"'capacity': (\d+)", outage_profile)
        
        # Erstellen eines Dictionaries mit den Werten in nebeneinander liegenden Spalten
        extracted_data = {}
        for i in range(len(start_times)):
            extracted_data[f'startTime_{i+1}'] = start_times[i]
            extracted_data[f'endTime_{i+1}'] = end_times[i]
            extracted_data[f'capacity_{i+1}'] = capacities[i]

        return extracted_data
    else:
        return {}

# Extrahieren der Werte aus der 'outageProfile'-Spalte
extracted_values = df['outageProfile'].apply(extract_values)

# Erstellen eines neuen DataFrames aus den extrahierten Werten
extracted_df = pd.DataFrame(extracted_values.tolist())

# Umwandlung der Spalten in den entsprechenden Datentyp
for col in extracted_df.columns:
    if 'startTime' in col or 'endTime' in col:
        extracted_df[col] = pd.to_datetime(extracted_df[col], errors='coerce')  # Fehlerhafte Einträge werden zu NaT
    elif 'capacity' in col:
        extracted_df[col] = extracted_df[col].astype(float)  # Oder int, je nach Bedarf

display(extracted_df.head(2))


,startTime_1,endTime_1,capacity_1,startTime_2,endTime_2,capacity_2,startTime_3,endTime_3,capacity_3,startTime_4,...,capacity_9,startTime_10,endTime_10,capacity_10,startTime_11,endTime_11,capacity_11,startTime_12,endTime_12,capacity_12
0,2020-11-26 09:00:00+00:00,2020-11-26 18:00:00+00:00,0.0,NaT,NaT,NaN,NaT,NaT,NaN,NaT,...,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
1,2020-12-11 06:00:00+00:00,2020-12-11 16:00:00+00:00,0.0,NaT,NaT,NaN,NaT,NaT,NaN,NaT,...,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN


In [85]:
df.shape

(436, 28)

In [86]:
# Gruppiere den DataFrame 'df' nach 'publishTime' und zähle die Anzahl der 'id'-Einträge in jeder Gruppe
df_publishTime = df.groupby(['publishTime'])['id'].count()

# Gebe die Liste der 'publishTime'-Indizes aus
print(df_publishTime.index.tolist())

# Sortiere die Ergebnisse in absteigender Reihenfolge
df_publishTime.sort_values(ascending=False)


[Timestamp('2020-01-29 15:52:00+0000', tz='UTC'), Timestamp('2020-01-31 13:04:00+0000', tz='UTC'), Timestamp('2020-02-02 06:40:00+0000', tz='UTC'), Timestamp('2020-02-02 07:11:00+0000', tz='UTC'), Timestamp('2020-02-02 08:44:00+0000', tz='UTC'), Timestamp('2020-02-03 11:19:00+0000', tz='UTC'), Timestamp('2020-02-06 11:03:00+0000', tz='UTC'), Timestamp('2020-02-10 18:21:00+0000', tz='UTC'), Timestamp('2020-02-13 11:06:00+0000', tz='UTC'), Timestamp('2020-02-14 09:43:00+0000', tz='UTC'), Timestamp('2020-02-15 15:06:00+0000', tz='UTC'), Timestamp('2020-02-17 15:05:00+0000', tz='UTC'), Timestamp('2020-02-24 10:50:00+0000', tz='UTC'), Timestamp('2020-02-25 14:39:00+0000', tz='UTC'), Timestamp('2020-02-28 13:14:00+0000', tz='UTC'), Timestamp('2020-02-28 13:18:00+0000', tz='UTC'), Timestamp('2020-03-04 23:16:00+0000', tz='UTC'), Timestamp('2020-03-04 23:22:00+0000', tz='UTC'), Timestamp('2020-03-05 15:02:00+0000', tz='UTC'), Timestamp('2020-03-06 12:51:00+0000', tz='UTC'), Timestamp('2020-03-

publishTime
2024-06-12 13:19:28+00:00    3
2024-06-13 18:37:32+00:00    3
2020-10-30 13:41:00+00:00    3
2023-07-17 10:42:04+00:00    3
2024-01-30 09:22:01+00:00    3
                            ..
2020-02-28 13:14:00+00:00    1
2020-02-28 13:18:00+00:00    1
2020-03-04 23:16:00+00:00    1
2020-03-04 23:22:00+00:00    1
2020-01-31 13:04:00+00:00    1
Name: id, Length: 403, dtype: int64

In [87]:
# Jede ID kommt nur einmal im Datensatz vor

# Gruppiere den DataFrame 'df' nach 'id' und zähle die Anzahl der 'dataset'-Einträge in jeder Gruppe
df_id = df.groupby(['id'])['dataset'].count()

# Sortiere die Ergebnisse in aufsteigender Reihenfolge
df_id.sort_values()

id
788801    1
788800    1
788392    1
788391    1
783622    1
         ..
19648     1
788816    1
788818    1
788819    1
17114     1
Name: dataset, Length: 436, dtype: int64

In [88]:
# In dieser Spalte ist nur  REMIT

# Gruppiere nach 'dataset' und zähle 'id'
df_dataset = df.groupby(['dataset'])['id'].count()
df_dataset

dataset
REMIT    436
Name: id, dtype: int64

In [89]:
# Gruppiere nach 'mrid' und zähle 'id'
df_mrid = df.groupby(['mrid'])['id'].count()
df_mrid

mrid
11XDONG-PT-----2-NGET-RMT-00000825    15
11XDONG-PT-----2-NGET-RMT-00000828     2
11XDONG-PT-----2-NGET-RMT-00000831    14
11XDONG-PT-----2-NGET-RMT-00000833     2
11XDONG-PT-----2-NGET-RMT-00000843     2
                                      ..
11XDONG-PT-----2-NGET-RMT-00002046     3
11XDONG-PT-----2-NGET-RMT-00002048     3
11XDONG-PT-----2-NGET-RMT-00002055     2
11XDONG-PT-----2-NGET-RMT-00002057     2
11XDONG-PT-----2-NGET-RMT-00002059     2
Name: id, Length: 151, dtype: int64

In [90]:
# Gruppiere nach 'revisionNumber' und zähle 'id'
df_revisionNumber = df.groupby(['revisionNumber'])['id'].count()

# Liste der eindeutigen 'revisionNumber'-Werte ausgibt
print(df_revisionNumber.index.tolist())
df_revisionNumber

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


revisionNumber
1     151
2     109
3      51
4      30
5      16
6      11
7       9
8       7
9       7
10      6
11      6
12      6
13      6
14      5
15      4
16      2
17      2
18      2
19      1
20      1
21      1
22      1
23      1
24      1
Name: id, dtype: int64

In [91]:
# Gruppiere nach 'publishTime' und zähle 'id'
df_publishTime = df.groupby(['publishTime'])['id'].count()

# Konvertiere den Index von 'df_publishTime' in eine Liste und gib sie aus
print(df_publishTime.index.tolist())

# Sortiere die gezählten Werte absteigend nach der Anzahl der 'id'
df_publishTime.sort_values(ascending=False)


[Timestamp('2020-01-29 15:52:00+0000', tz='UTC'), Timestamp('2020-01-31 13:04:00+0000', tz='UTC'), Timestamp('2020-02-02 06:40:00+0000', tz='UTC'), Timestamp('2020-02-02 07:11:00+0000', tz='UTC'), Timestamp('2020-02-02 08:44:00+0000', tz='UTC'), Timestamp('2020-02-03 11:19:00+0000', tz='UTC'), Timestamp('2020-02-06 11:03:00+0000', tz='UTC'), Timestamp('2020-02-10 18:21:00+0000', tz='UTC'), Timestamp('2020-02-13 11:06:00+0000', tz='UTC'), Timestamp('2020-02-14 09:43:00+0000', tz='UTC'), Timestamp('2020-02-15 15:06:00+0000', tz='UTC'), Timestamp('2020-02-17 15:05:00+0000', tz='UTC'), Timestamp('2020-02-24 10:50:00+0000', tz='UTC'), Timestamp('2020-02-25 14:39:00+0000', tz='UTC'), Timestamp('2020-02-28 13:14:00+0000', tz='UTC'), Timestamp('2020-02-28 13:18:00+0000', tz='UTC'), Timestamp('2020-03-04 23:16:00+0000', tz='UTC'), Timestamp('2020-03-04 23:22:00+0000', tz='UTC'), Timestamp('2020-03-05 15:02:00+0000', tz='UTC'), Timestamp('2020-03-06 12:51:00+0000', tz='UTC'), Timestamp('2020-03-

publishTime
2024-06-12 13:19:28+00:00    3
2024-06-13 18:37:32+00:00    3
2020-10-30 13:41:00+00:00    3
2023-07-17 10:42:04+00:00    3
2024-01-30 09:22:01+00:00    3
                            ..
2020-02-28 13:14:00+00:00    1
2020-02-28 13:18:00+00:00    1
2020-03-04 23:16:00+00:00    1
2020-03-04 23:22:00+00:00    1
2020-01-31 13:04:00+00:00    1
Name: id, Length: 403, dtype: int64

In [92]:
# Gruppiere nach 'createdTime' und zähle 'id' pro Gruppe
df_createdTime = df.groupby(['createdTime'])['id'].count()

# Konvertiere den Index von 'createdTime' in eine Liste und gib sie aus
print(df_createdTime.index.tolist())

# Sortiere die Gruppen basierend auf der Anzahl der 'id' absteigend
df_createdTime.sort_values(ascending=False)


[Timestamp('2020-01-29 15:52:00+0000', tz='UTC'), Timestamp('2020-01-31 13:04:00+0000', tz='UTC'), Timestamp('2020-02-02 06:40:00+0000', tz='UTC'), Timestamp('2020-02-02 07:11:00+0000', tz='UTC'), Timestamp('2020-02-02 08:44:00+0000', tz='UTC'), Timestamp('2020-02-03 11:19:00+0000', tz='UTC'), Timestamp('2020-02-06 11:03:00+0000', tz='UTC'), Timestamp('2020-02-10 18:21:00+0000', tz='UTC'), Timestamp('2020-02-13 11:06:00+0000', tz='UTC'), Timestamp('2020-02-14 09:43:00+0000', tz='UTC'), Timestamp('2020-02-15 15:06:00+0000', tz='UTC'), Timestamp('2020-02-17 15:05:00+0000', tz='UTC'), Timestamp('2020-02-24 10:50:00+0000', tz='UTC'), Timestamp('2020-02-25 14:39:00+0000', tz='UTC'), Timestamp('2020-02-28 13:14:00+0000', tz='UTC'), Timestamp('2020-02-28 13:18:00+0000', tz='UTC'), Timestamp('2020-03-04 23:16:00+0000', tz='UTC'), Timestamp('2020-03-04 23:22:00+0000', tz='UTC'), Timestamp('2020-03-05 15:02:00+0000', tz='UTC'), Timestamp('2020-03-06 12:51:00+0000', tz='UTC'), Timestamp('2020-03-

createdTime
2020-10-30 13:41:00+00:00    3
2020-07-06 19:24:00+00:00    2
2020-10-30 23:41:00+00:00    2
2020-10-30 17:03:00+00:00    2
2024-06-13 18:31:00+00:00    2
                            ..
2020-03-09 08:56:00+00:00    1
2020-03-12 06:38:00+00:00    1
2020-03-12 18:56:00+00:00    1
2020-03-13 09:36:00+00:00    1
2024-09-29 19:05:00+00:00    1
Name: id, Length: 426, dtype: int64

In [93]:
# Datumsspalten in datetime umwandeln
df['publishTime'] = pd.to_datetime(df['publishTime'])
df['createdTime'] = pd.to_datetime(df['createdTime'])

# Häufigkeit der Datumswerte zählen
häufigkeit_publishTime = df['publishTime'].value_counts().sort_index()
häufigkeit_createdTime = df['createdTime'].value_counts().sort_index()

# Plotly-Figuren erstellen
fig = go.Figure()

# Punkte für publishTime hinzufügen
fig.add_trace(go.Scatter(
    x=häufigkeit_publishTime.index,
    y=häufigkeit_publishTime.values,
    mode='markers',
    name='publishTime'
))

# Punkte für createdTime hinzufügen
fig.add_trace(go.Scatter(
    x=häufigkeit_createdTime.index,
    y=häufigkeit_createdTime.values,
    mode='markers',
    name='createdTime'
))

# Layout anpassen
fig.update_layout(
    title='Häufigkeit der Ausfälle',
    xaxis_title='Datum und Uhrzeit',
    yaxis_title='Häufigkeit',
    xaxis=dict(
        tickformat='%Y-%m-%d %H:%M:%S',
        tickangle=45,
        rangeslider_visible=True
    ),
    hovermode='x unified'
)

# Diagramm anzeigen
fig.show()


In [94]:
# Gruppiere nach 'publishTime' und zähle 'id' pro Gruppe
df_publishTime = df.groupby(['publishTime'])['id'].count()

# Konvertiere den Index von 'publishTime' in eine Liste und gib sie aus
print(df_publishTime.index.tolist())

# Ergebnis nach der Anzahl der 'publishTime' absteigend sortieren
df_publishTime.sort_values(ascending=False)


[Timestamp('2020-01-29 15:52:00+0000', tz='UTC'), Timestamp('2020-01-31 13:04:00+0000', tz='UTC'), Timestamp('2020-02-02 06:40:00+0000', tz='UTC'), Timestamp('2020-02-02 07:11:00+0000', tz='UTC'), Timestamp('2020-02-02 08:44:00+0000', tz='UTC'), Timestamp('2020-02-03 11:19:00+0000', tz='UTC'), Timestamp('2020-02-06 11:03:00+0000', tz='UTC'), Timestamp('2020-02-10 18:21:00+0000', tz='UTC'), Timestamp('2020-02-13 11:06:00+0000', tz='UTC'), Timestamp('2020-02-14 09:43:00+0000', tz='UTC'), Timestamp('2020-02-15 15:06:00+0000', tz='UTC'), Timestamp('2020-02-17 15:05:00+0000', tz='UTC'), Timestamp('2020-02-24 10:50:00+0000', tz='UTC'), Timestamp('2020-02-25 14:39:00+0000', tz='UTC'), Timestamp('2020-02-28 13:14:00+0000', tz='UTC'), Timestamp('2020-02-28 13:18:00+0000', tz='UTC'), Timestamp('2020-03-04 23:16:00+0000', tz='UTC'), Timestamp('2020-03-04 23:22:00+0000', tz='UTC'), Timestamp('2020-03-05 15:02:00+0000', tz='UTC'), Timestamp('2020-03-06 12:51:00+0000', tz='UTC'), Timestamp('2020-03-

publishTime
2024-06-12 13:19:28+00:00    3
2024-06-13 18:37:32+00:00    3
2020-10-30 13:41:00+00:00    3
2023-07-17 10:42:04+00:00    3
2024-01-30 09:22:01+00:00    3
                            ..
2020-02-28 13:14:00+00:00    1
2020-02-28 13:18:00+00:00    1
2020-03-04 23:16:00+00:00    1
2020-03-04 23:22:00+00:00    1
2020-01-31 13:04:00+00:00    1
Name: id, Length: 403, dtype: int64

In [95]:
# In dieser Spalte ist nur UnavailabilitiesOfElectricityFacilities

# Gruppiere nach 'messageType' und zähle 'id' pro Gruppe
df_messageType = df.groupby(['messageType'])['id'].count()

# Konvertiere den Index von 'messageType' in eine Liste und gib sie aus
print(df_messageType.index.tolist())

df_messageType

['UnavailabilitiesOfElectricityFacilities']


messageType
UnavailabilitiesOfElectricityFacilities    436
Name: id, dtype: int64

In [96]:
# Ist dieser Spalte ist nur  messageHeading

# Gruppiere nach 'messageHeading' und zähle 'id' pro Gruppe
df_messageHeading = df.groupby(['messageHeading'])['id'].count()

# Konvertiere den Index von 'messageHeading' in eine Liste und gib sie aus
print(df_messageHeading.index.tolist())

df_messageHeading

['REMIT Information']


messageHeading
REMIT Information    436
Name: id, dtype: int64

In [97]:
# Ist dieser Spalte ist nur Production unavailability

# Gruppiere nach 'eventType' und zähle 'id' pro Gruppe
df_eventType = df.groupby(['eventType'])['id'].count()

# Konvertiere den Index von 'eventType' in eine Liste und gib sie aus
print(df_eventType.index.tolist())

df_eventType

['Production unavailability']


eventType
Production unavailability    436
Name: id, dtype: int64

hier weitermachen

In [98]:
# Gruppiere nach 'unavailabilityType' und zähle 'id' pro Gruppe
df_unavailabilityType = df.groupby(['unavailabilityType'])['id'].count()

# Konvertiere den Index von 'unavailabilityType' in eine Liste und gib sie aus
print(df_unavailabilityType.index.tolist())

df_unavailabilityType

['Planned', 'Unplanned']


unavailabilityType
Planned      348
Unplanned     88
Name: id, dtype: int64

In [99]:
# Ist dieser Spalte ist nur DONG013

# Gruppiere nach 'participantId' und zähle 'id' pro Gruppe
df_participantId = df.groupby(['participantId'])['id'].count()

# Konvertiere den Index von 'participantId' in eine Liste und gib sie aus
print(df_participantId.index.tolist())

df_participantId

['DONG013']


participantId
DONG013    436
Name: id, dtype: int64

In [100]:
# Ist dieser Spalte ist nur 11XDONG-PT-----2

# Gruppiere nach 'registrationCode' und zähle 'id' pro Gruppe
df_registrationCode = df.groupby(['registrationCode'])['id'].count()

# Konvertiere den Index von 'registrationCode' in eine Liste und gib sie aus
print(df_registrationCode.index.tolist())

df_registrationCode

['11XDONG-PT-----2']


registrationCode
11XDONG-PT-----2    436
Name: id, dtype: int64

In [101]:
# Gruppiere nach 'assetId' und zähle 'id' pro Gruppe
df_assetId = df.groupby(['assetId'])['id'].count()

# Konvertiere den Index von 'assetId' in eine Liste und gib sie aus
print(df_assetId.index.tolist())
df_assetId


['T_HOWAO-1', 'T_HOWAO-2', 'T_HOWAO-3']


assetId
T_HOWAO-1    145
T_HOWAO-2    133
T_HOWAO-3    158
Name: id, dtype: int64

In [102]:
df['counter_1'] = 1

# 'publishTime' in datetime umwandeln und auf das Datum reduzieren
df['publishTime'] = pd.to_datetime(df['publishTime']).dt.date

# Daten aggregieren (z.B. durch Summieren von 'counter_1' pro Tag und 'assetId')
df_aggregated = df.groupby(['publishTime', 'assetId'], as_index=False).agg({'counter_1': 'sum'})

# Balkendiagramm erstellen
fig = px.bar(df_aggregated, x='publishTime', y='counter_1', color='assetId',
             title='Ausfälle der verschiedenen Windanlagen (Tagesbasis)',
             )

# Range Slider hinzufügen
fig.update_layout(
    xaxis=dict(
        title='Datum',
        rangeslider=dict(visible=True),
        type='date'
    ),
    yaxis_title='Anzahl',
    title_x=0.5,  # Titel zentrieren
)

# Diagramm anzeigen
fig.show()

         


In [103]:
# Gruppiere nach 'publishTime' und zähle 'id' pro Gruppe
df_publishTime = df.groupby(['publishTime'])['id'].count()

# Konvertiere den Index von 'publishTime' in eine Liste und gib sie aus
print(df_publishTime.index.tolist())

# Ergebnis nach der Anzahl der 'publishTime' absteigend sortieren
df_publishTime.sort_values(ascending=False)


[datetime.date(2020, 1, 29), datetime.date(2020, 1, 31), datetime.date(2020, 2, 2), datetime.date(2020, 2, 3), datetime.date(2020, 2, 6), datetime.date(2020, 2, 10), datetime.date(2020, 2, 13), datetime.date(2020, 2, 14), datetime.date(2020, 2, 15), datetime.date(2020, 2, 17), datetime.date(2020, 2, 24), datetime.date(2020, 2, 25), datetime.date(2020, 2, 28), datetime.date(2020, 3, 4), datetime.date(2020, 3, 5), datetime.date(2020, 3, 6), datetime.date(2020, 3, 9), datetime.date(2020, 3, 12), datetime.date(2020, 3, 13), datetime.date(2020, 3, 14), datetime.date(2020, 3, 19), datetime.date(2020, 3, 20), datetime.date(2020, 3, 24), datetime.date(2020, 3, 27), datetime.date(2020, 3, 30), datetime.date(2020, 4, 2), datetime.date(2020, 4, 3), datetime.date(2020, 4, 4), datetime.date(2020, 4, 6), datetime.date(2020, 4, 7), datetime.date(2020, 4, 8), datetime.date(2020, 4, 18), datetime.date(2020, 4, 19), datetime.date(2020, 4, 20), datetime.date(2020, 4, 21), datetime.date(2020, 4, 24), date

publishTime
2020-10-30    17
2021-03-20    11
2020-07-06    10
2020-04-21    10
2023-04-24     9
              ..
2024-05-21     1
2024-05-27     1
2024-06-03     1
2024-06-01     1
2024-06-04     1
Name: id, Length: 188, dtype: int64

In [104]:
# Ist dieser Spalte ist nur Production

# Gruppiere nach 'assetType' und zähle 'id' pro Gruppe
df_assetType = df.groupby(['assetType'])['id'].count()

# Konvertiere den Index von 'assetType' in eine Liste und gib sie aus
print(df_assetType.index.tolist())

df_assetType

['Production']


assetType
Production    436
Name: id, dtype: int64

In [105]:
# Gruppiere nach 'affectedUnit' und zähle 'id' pro Gruppe
df_affectedUnit = df.groupby(['affectedUnit'])['id'].count()

# Konvertiere den Index von 'affectedUnit' in eine Liste und gib sie aus
print(df_affectedUnit.index.tolist())

df_affectedUnit

['HOWAO-1', 'HOWAO-2', 'HOWAO-3']


affectedUnit
HOWAO-1    145
HOWAO-2    133
HOWAO-3    158
Name: id, dtype: int64

In [106]:
# Gruppiere nach 'affectedUnitEIC' und zähle 'id' pro Gruppe
df_affectedUnitEIC = df.groupby(['affectedUnitEIC'])['id'].count()

# Konvertiere den Index von 'affectedUnitEIC' in eine Liste und gib sie aus
print(df_affectedUnitEIC.index.tolist())

df_affectedUnitEIC

['48W00000HOWAO-1M', '48W00000HOWAO-2K', '48W00000HOWAO-3I']


affectedUnitEIC
48W00000HOWAO-1M    145
48W00000HOWAO-2K    133
48W00000HOWAO-3I    158
Name: id, dtype: int64

In [107]:
# Ist dieser Spalte ist nur B7

# Gruppiere nach 'affectedArea' und zähle 'id' pro Gruppe
df_affectedArea = df.groupby(['affectedArea'])['id'].count()

# Konvertiere den Index von 'affectedArea' in eine Liste und gib sie aus
print(df_affectedArea.index.tolist())

df_affectedArea

['B7']


affectedArea
B7    436
Name: id, dtype: int64

In [108]:
# Ist dieser Spalte ist nur 10YGB----------A

# Gruppiere nach 'biddingZone' und zähle 'id' pro Gruppe
df_biddingZone = df.groupby(['biddingZone'])['id'].count()

# Liste der eindeutigen 'biddingZone'-Werte ausgeben
print(df_biddingZone.index.tolist())

df_biddingZone

['10YGB----------A']


biddingZone
10YGB----------A    436
Name: id, dtype: int64

In [109]:
# Ist dieser Spalte ist nur Wind Offshore

# Gruppiere nach 'fuelType' und zähle 'id' pro Gruppe
df_fuelType = df.groupby(['fuelType'])['id'].count()

# Liste der eindeutigen 'fuelType'-Werte ausgeben
print(df_fuelType.index.tolist())

df_fuelType

['Wind Offshore']


fuelType
Wind Offshore    436
Name: id, dtype: int64

In [110]:
# Gruppiere nach 'normalCapacity' und zähle 'id' pro Gruppe
df_normalCapacity = df.groupby(['normalCapacity'])['id'].count()

# Liste der eindeutigen 'normalCapacity'-Werte ausgeben
print(df_normalCapacity.index.tolist())

df_normalCapacity

[400, 406]


normalCapacity
400    431
406      5
Name: id, dtype: int64

In [111]:
# Gruppiere nach 'availableCapacity' und zähle 'id' pro Gruppe
df_availableCapacity = df.groupby(['availableCapacity'])['id'].count()

# Liste der eindeutigen 'availableCapacity'-Werte ausgeben
print(df_availableCapacity.index.tolist())

df_availableCapacity

[0, 75, 77, 98, 100, 105, 140, 150, 161, 168, 173, 175, 182, 189, 190, 196, 200, 203, 224, 238, 240, 245, 250, 252, 259, 260, 266, 273, 280, 287, 294, 300]


availableCapacity
0      201
75      11
77       1
98       7
100      3
105      2
140      5
150     14
161      2
168      2
173      2
175      2
182      1
189      2
190      1
196      2
200     44
203      2
224      2
238      2
240     12
245      6
250     33
252      2
259      1
260      5
266     17
273     11
280     13
287     10
294     17
300      1
Name: id, dtype: int64

In [112]:
# 'publishTime' in datetime umwandeln und auf das Datum reduzieren
df['publishTime'] = pd.to_datetime(df['publishTime']).dt.date

# Daten aggregieren (z.B. durch Summieren von 't' pro Tag und 'availableCapacity')
df_filter = df.filter(['publishTime', 'availableCapacity','counter_1'])

df_aggregated = df_filter.groupby(['publishTime', 'availableCapacity'], as_index=False).agg({'counter_1': 'count'})

# Sortieren der availableCapacity-Werte
available_capacities_sorted = sorted(df_aggregated['availableCapacity'].unique())

# Leere Figur erstellen
fig = go.Figure()

# Dropdown-Buttons-Liste initialisieren
dropdown_buttons = []

# Eine Spur pro unique availableCapacity hinzufügen
for i, available_capacity in enumerate(available_capacities_sorted):
    # Daten für diese bestimmte availableCapacity filtern
    filtered_df = df_aggregated[df_aggregated['availableCapacity'] == available_capacity]
    
    # Spur für diese availableCapacity hinzufügen
    fig.add_trace(go.Bar(
        x=filtered_df['publishTime'],
        y=filtered_df['counter_1'],
        name=f'availableCapacity: {available_capacity}',
        visible=False  # Alle Spuren zunächst unsichtbar machen
    ))
    
    # Dropdown-Option für diese availableCapacity hinzufügen
    dropdown_buttons.append({
        'label': f'availableCapacity: {available_capacity}',
        'method': 'update',
        'args': [{'visible': [i == j for j in range(len(available_capacities_sorted))]}]  # Nur die gewählte Spur sichtbar machen
    })

# "Alle Kapazitäten" Button hinzufügen, um alle Spuren sichtbar zu machen
dropdown_buttons.insert(0, {
    'label': 'Alle Kapazitäten',
    'method': 'update',
    'args': [{'visible': [True] * len(available_capacities_sorted)}]
})

# Die erste Spur standardmäßig sichtbar machen
fig.data[0].visible = True

# Layout mit Dropdown-Menü, Range Slider und Zeitintervall-Buttons (Woche, Monat, Quartal, Jahr)
fig.update_layout(
    title='availableCapacity',
    xaxis=dict(
        title='Datum',
        rangeslider=dict(visible=True),
        type='date',
        rangeselector=dict(
            buttons=list([
                dict(count=7, label='Woche', step='day', stepmode='backward'),
                dict(count=1, label='Monat', step='month', stepmode='backward'),
                dict(count=3, label='Quartal', step='month', stepmode='backward'),  # Quartals-Button
                dict(count=1, label='Jahr', step='year', stepmode='backward'),
                dict(step='all')  # Zeigt den gesamten Zeitraum an
            ])
        )
    ),
    yaxis_title='Anzahl',
    yaxis=dict(
        title='Anzahl',
        autorange=False,  # Autorange deaktivieren
        range=[0, df_aggregated['counter_1'].max() * 1.1]  # Manuelle Range für die Y-Achse (10% mehr als max Wert)
    ),
    title_x=0.5,  # Titel zentrieren
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        'x': 1.15,  # Position rechts vom Diagramm
        'xanchor': 'left',
        'y': 1.15,
        'yanchor': 'top'
    }]
)

# Diagramm anzeigen
fig.show()



In [113]:
# Gruppiere nach 'unavailableCapacity' und zähle 'id' pro Gruppe
df_unavailableCapacity = df.groupby(['unavailableCapacity'])['id'].count()

# Liste der eindeutigen 'unavailableCapacity'-Werte ausgeben
# print(df_unavailableCapacity.index.tolist())

df_unavailableCapacity

unavailableCapacity
100      1
106     17
113     10
120     13
127     11
134     17
140      5
141      1
148      2
150     33
155      6
160      9
162      2
166      3
176      2
200     44
203      2
204      2
210      1
211      2
218      1
225      2
227      2
232      2
239      2
250     14
260      5
295      2
300      3
302      7
323      1
325     11
400    201
Name: id, dtype: int64

In [114]:
# Gruppiere nach 'eventStartTime' und zähle 'id' pro Gruppe
df_eventStartTime = df.groupby(['eventStartTime'])['id'].count()

# Liste der eindeutigen 'eventStartTime'-Werte ausgeben
print(df_eventStartTime.index.tolist())

# Ergebnis nach der Anzahl der 'eventStartTime' absteigend sortieren
df_eventStartTime.sort_values(ascending=False)


[Timestamp('2020-01-30 06:00:00+0000', tz='UTC'), Timestamp('2020-03-04 06:00:00+0000', tz='UTC'), Timestamp('2020-03-04 22:30:00+0000', tz='UTC'), Timestamp('2020-03-12 06:00:00+0000', tz='UTC'), Timestamp('2020-03-24 06:00:00+0000', tz='UTC'), Timestamp('2020-04-03 06:00:00+0000', tz='UTC'), Timestamp('2020-04-03 08:00:00+0000', tz='UTC'), Timestamp('2020-04-06 08:30:00+0000', tz='UTC'), Timestamp('2020-04-07 06:30:00+0000', tz='UTC'), Timestamp('2020-04-07 08:30:00+0000', tz='UTC'), Timestamp('2020-04-08 06:30:00+0000', tz='UTC'), Timestamp('2020-04-18 18:30:00+0000', tz='UTC'), Timestamp('2020-04-19 00:00:00+0000', tz='UTC'), Timestamp('2020-04-21 08:00:00+0000', tz='UTC'), Timestamp('2020-04-21 10:30:00+0000', tz='UTC'), Timestamp('2020-04-21 12:00:00+0000', tz='UTC'), Timestamp('2020-04-21 12:30:00+0000', tz='UTC'), Timestamp('2020-04-21 13:30:00+0000', tz='UTC'), Timestamp('2020-04-21 14:00:00+0000', tz='UTC'), Timestamp('2020-04-26 11:00:00+0000', tz='UTC'), Timestamp('2020-05-

eventStartTime
2024-01-23 11:20:00+00:00    24
2024-01-19 16:55:00+00:00    16
2020-01-30 06:00:00+00:00    15
2024-01-29 02:10:00+00:00    15
2020-10-30 11:00:00+00:00    15
                             ..
2024-05-31 11:30:00+00:00     1
2024-05-04 08:00:00+00:00     1
2024-05-03 08:00:00+00:00     1
2024-05-31 10:00:00+00:00     1
2024-05-09 08:00:00+00:00     1
Name: id, Length: 122, dtype: int64

In [115]:
# Gruppiere nach 'eventEndTime' und zähle 'id' pro Gruppe
df_eventEndTime = df.groupby(['eventEndTime'])['id'].count()

# Liste der eindeutigen 'eventEndTime'-Werte ausgeben
print(df_eventEndTime.index.tolist())

# Ergebnis nach der Anzahl der 'eventEndTime' absteigend sortieren
df_eventEndTime.sort_values(ascending=False)


['2020-02-01T15:00:00Z', '2020-02-02T18:00:00Z', '2020-02-03T02:00:00Z', '2020-02-05T02:00:00Z', '2020-02-07T18:00:00Z', '2020-02-12T20:00:00Z', '2020-02-13T19:30:00Z', '2020-02-17T12:00:00Z', '2020-02-21T16:00:00Z', '2020-02-22T19:30:00Z', '2020-02-28T12:00:00Z', '2020-03-02T12:00:00Z', '2020-03-04T06:00:00Z', '2020-03-04T22:30:00Z', '2020-03-09T16:00:00Z', '2020-03-12T19:00:00Z', '2020-03-14T06:00:00Z', '2020-03-15T16:00:00Z', '2020-03-18T16:00:00Z', '2020-03-19T16:00:00Z', '2020-03-26T22:00:00Z', '2020-03-28T16:00:00Z', '2020-03-30T15:00:00Z', '2020-03-31T15:00:00Z', '2020-04-02T14:00:00Z', '2020-04-02T16:00:00Z', '2020-04-03T10:00:00Z', '2020-04-03T12:00:00Z', '2020-04-06T14:30:00Z', '2020-04-07T12:30:00Z', '2020-04-07T14:30:00Z', '2020-04-08T13:00:00Z', '2020-04-08T14:30:00Z', '2020-04-19T00:00:00Z', '2020-04-19T11:00:00Z', '2020-04-20T11:00:00Z', '2020-04-21T10:00:00Z', '2020-04-21T10:30:00Z', '2020-04-21T11:30:00Z', '2020-04-21T12:00:00Z', '2020-04-21T12:30:00Z', '2020-04-21T14:

eventEndTime
2024-01-31T16:00:00Z    12
2024-06-12T16:00:00Z     6
2021-01-15T16:00:00Z     6
2024-01-27T16:00:00Z     5
2024-02-25T12:00:00Z     5
                        ..
2024-05-31T14:30:00Z     1
2024-06-01T14:30:00Z     1
2020-02-05T02:00:00Z     1
2020-02-02T18:00:00Z     1
2020-02-01T15:00:00Z     1
Name: id, Length: 263, dtype: int64

In [116]:
# Umwandlung der Zeitspalten in naive Datetime
df['eventStartTime'] = pd.to_datetime(df['eventStartTime'])
df['eventEndTime'] = pd.to_datetime(df['eventEndTime'])

# Beispielhafte Start- und Endpunkte
startpunkte = df['eventStartTime']
endpunkte = df['eventEndTime']

# Erstellen einer leeren Figur
fig = go.Figure()

# Schleife zum Hochzählen der Y-Werte
for i, (start, end) in enumerate(zip(startpunkte, endpunkte), start=1):
    fig.add_trace(go.Scatter(
        x=[start, end],  # X-Werte sind die Start- und Endzeiten
        y=[i, i],  # Beide Y-Werte sind gleich, damit eine horizontale Linie entsteht
        mode='lines+markers',
        name=f'Linie {i}'
    ))

# Layout anpassen und Range-Slider hinzufügen
fig.update_layout(
    title='Horizontale Linien mit aufsteigenden Werten',
    xaxis=dict(
        title='Datum',
        rangeslider=dict(visible=True),  # Range-Slider sichtbar machen
        type='date'
    ),
    yaxis_title='Werte',
    title_x=0.5,  # Titel zentrieren
)

# Anzeige der Figur
fig.show()


In [117]:
# Gruppiere nach 'cause' und zähle 'id' pro Gruppe
df_cause= df.groupby(['cause'])['id'].count()

# Liste der eindeutigen 'cause'-Werte ausgeben
print(df_cause.index.tolist())

# Ergebnis nach der Anzahl der 'cause' absteigend sortieren
df_cause.sort_values(ascending=False)

['Forced Outage', 'NonOperator Alert', 'Planned Outage', 'Turbine', 'Under Investigation']


cause
Planned Outage         342
Under Investigation     86
NonOperator Alert        3
Turbine                  3
Forced Outage            2
Name: id, dtype: int64